# Callbacks in NS-3 with Python bindings

First, let's import the bindings of NS-3.

In [ ]:
from ns import ns

In the ns-3 network simulator, events play a crucial role. Think of events as specific triggers that cause changes within the simulator's state machine, helping to simulate different situations and conditions in a network.

Here's a simple way to understand events:

### Events
These are like alarms set for specific times to perform specific tasks. In ns-3, they help in changing the state or condition within the simulation.

### Tuple
Events are generally made up of a tuple, which is a pair of related elements. In this case, the tuple contains a delay and a function pointer.
- **Delay**: This is the time difference between the current time (ns.Simulator.Now() in Python) and the future time when the event will be activated or dispatched. It tells the simulator when to execute the event.
- **Function Pointer**: This points to the function that will be executed when the event is dispatched. It tells the simulator what to do when the event time arrives.

So, in essence, an event in ns-3 tells the simulator what to do (function pointer) and when to do it (delay). This mechanism helps in effectively simulating various network scenarios and observing their behaviors.

## Simple Events

To understand how events work in ns-3 with Python bindings, let's begin with a simple exercise. We will create a function that will be called by an event.
This function's job is just to print the current virtual time of the simulator.

Here are the steps to follow:

1. Create a Function
First, define a function. Let’s name it print_virtual_time. This function will not take any parameters.
Inside the function, use ns.Simulator.Now() to get the current virtual time of the simulator.
Print the obtained virtual time.

1. Schedule an Event
Now, schedule an event to call the print_virtual_time function after a certain delay. Use ns.Simulator.Schedule() to schedule the event.
Provide the delay and the function name as parameters.

1. Run the Simulator
Finally, run the simulator using ns.Simulator.Run() to see the output.
You will see the printed virtual time after the delay you set.

By following these steps, you have created an event that triggers a function to print the current virtual time of the simulator. This is a basic example to help you understand how events and scheduling work in ns-3 using Python bindings.


Here's how you can write the function in Python:

In [ ]:
def print_virtual_time() -> None:
    print(f"The current virtual time: {ns.Simulator.Now()}")

Although there is a function to schedule an event in ns-3 that would look for our example such as follows (we schedule a function call after 5 seconds):

```python
ns.Simulator.Schedule(ns.Seconds(5), print_virtual_time)
```

It will not work as Python callable is not accepted by the C++ code of the simulator. To overcome this, we can use a following function:

In [ ]:
ns.cppyy.cppdef("""
    EventImpl* mdsMakeEvent(void (*f)())
    {
        return MakeEvent(f);
    }
""")


Now, we can use the newly defined C++ function to convert our Python callable so that it can be used in ns-3. Don't forget to use correct namespace of cppyy library.

In [ ]:
ns.cppyy.gbl.mdsMakeEvent(print_virtual_time)

For more information on EventImpl class, please visit ns-3 documentation: https://www.nsnam.org/docs/release/3.39/doxygen/d7/da2/classns3_1_1_event_impl.html

In [ ]:
# Print current time (0s)
print_virtual_time()

for i in range(1, 10):
    # Create an event for each second of the simulation
    event = ns.cppyy.gbl.mdsMakeEvent(print_virtual_time)
    ns.Simulator.Schedule(ns.Seconds(i), event)

# Set the simulation to stop after 15 seconds
ns.Simulator.Stop(ns.Seconds(15))

# Run the simulator
ns.Simulator.Run()

# And destroy it after simulation finishes
ns.Simulator.Destroy()

## Task

1. Modify the code so that it displays virtual time in seconds.

## Repeated events

We can modify the transformation function to schedule a repeated events for us.

In [ ]:
ns.cppyy.cppdef("""
    EventImpl* mdsRepeatedMakeEvent(void (*f)(ns3::Time&), ns3::Time period)
    {
        return MakeEvent(f, period);
    }
""")

In [ ]:
def print_virtual_time_rep(period: ns.Time) -> None:
    print(f"The current virtual time: {ns.Simulator.Now()}")
    event = ns.cppyy.gbl.mdsRepeatedMakeEvent(print_virtual_time_rep, period)
    ns.Simulator.Schedule(period, event)

In [ ]:
# Print current time (0s)
print_virtual_time()

# Create an event for each second of the simulation
event = ns.cppyy.gbl.mdsRepeatedMakeEvent(print_virtual_time_rep, ns.Seconds(1))
ns.Simulator.Schedule(ns.Seconds(1), event)

# Set the simulation to stop after 15 seconds
ns.Simulator.Stop(ns.Seconds(15))

# Run the simulator
ns.Simulator.Run()

# And destroy it after simulation finishes
ns.Simulator.Destroy()

Now, we only schedule the first event and all the other events are scheduled automatically using the recursive function call.

## Adding randomness to simulation

We can use events to change the parameters of simulation on the fly. Now, for instance, we can change the packet size of the UDP datagrams generated by UdpEchoClient application.

In [ ]:
ns.cppyy.cppdef("""
    EventImpl* mdsPacketSizeChangeMakeEvent(void (*f)(Ptr<UdpEchoClient>), Ptr<UdpEchoClient> cli)
    {
        return MakeEvent(f, cli);
    }
""")

In [ ]:
import random
from typing import Union

def change_packet_size_event(echo_client: Union[ns.UdpEchoClient, ns.Ptr]) -> None:
    try:
        echo_client = echo_client.__deref__()
    except AttributeError as e:
        # already a UdpEchoClient
        pass
    echo_client.SetDataSize(random.randint(100, 1000))
    event = ns.cppyy.gbl.mdsPacketSizeChangeMakeEvent(change_packet_size_event, echo_client)
    ns.Simulator.Schedule(ns.Seconds(1), event)



In [ ]:
ns.LogComponentEnable("UdpEchoClientApplication", ns.core.LOG_LEVEL_INFO)
ns.LogComponentEnable("UdpEchoServerApplication", ns.core.LOG_LEVEL_INFO)

In [ ]:
nodes = ns.network.NodeContainer()
nodes.Create(2)

p2p = ns.point_to_point.PointToPointHelper()
p2p.SetDeviceAttribute("DataRate", ns.core.StringValue("5Mbps"))
p2p.SetChannelAttribute("Delay", ns.core.StringValue("2ms"))

devices = p2p.Install(nodes)

ip_stack = ns.internet.InternetStackHelper()
ip_stack.Install(nodes)

addr = ns.internet.Ipv4AddressHelper()
addr.SetBase(ns.network.Ipv4Address("10.0.0.0"),
                ns.network.Ipv4Mask("255.255.255.0"))

ifaces = addr.Assign(devices)

ECHO_PORT = 9
echo_srv_helper = ns.UdpEchoServerHelper(ECHO_PORT)
srv_apps = echo_srv_helper.Install(nodes.Get(1))
srv_addr = ifaces.GetAddress(1).ConvertTo()

echo_client_helper = ns.applications.UdpEchoClientHelper(srv_addr, ECHO_PORT)
client_apps = echo_client_helper.Install(nodes.Get(0))
print(f"Client apps: {client_apps.GetN()}")

attrs = {
    "MaxPackets": ns.UintegerValue,
    "Interval": ns.TimeValue,
    "PacketSize": ns.UintegerValue,
}
for attr, f in attrs.items():
    f = f()
    client_apps.Get(0).__deref__().GetAttribute(attr, f)
    print(f"\t{attr}: {f.Get()}")

# Schedule packet size change
ec = client_apps.Get(0).GetObject[ns.UdpEchoClient]()
event = ns.cppyy.gbl.mdsPacketSizeChangeMakeEvent(change_packet_size_event, ec)
ns.Simulator.Schedule(ns.Seconds(1), event)

## Time the simulation

srv_apps.Start(ns.core.Seconds(1.0))
srv_apps.Stop(ns.core.Seconds(20.0))
client_apps.Start(ns.core.Seconds(2.0)) # has to be later than server
client_apps.Stop(ns.core.Seconds(20.0))

## Run the simulator

ns.Simulator.Stop(ns.Seconds(20))
ns.Simulator.Run()
ns.Simulator.Destroy()